In [25]:
import os
import torch
import sys

parent_dir = os.path.abspath("..")
sys.path.append(parent_dir)


from src.classifying import (
    ActivationsHandler,
    get_correctness_direction_classifier
)

In [29]:
folder_path = "./activations/activations_trivia_1000_4000_right/layer_12"  
activation_files = sorted([f for f in os.listdir(folder_path) if f.endswith(".pt")])
activations_list_true = [torch.load(os.path.join(folder_path, f), map_location='cpu').cpu() for f in activation_files]

folder_path = "./activations/activations_trivia_1000_4000_wrong/layer_12"
activation_files = sorted([f for f in os.listdir(folder_path) if f.endswith(".pt")])
activations_list_false = [torch.load(os.path.join(folder_path, f), map_location='cpu').cpu() for f in activation_files]

In [31]:
activations_true = torch.cat(activations_list_true, dim=0)
activations_false = torch.cat(activations_list_false, dim=0)
activations_tensor_train = torch.cat([activations_true[:2000], activations_false[:2000]], dim=0)
activations_tensor_test = torch.cat([activations_true[2000:2500], activations_false[2000:2500]], dim=0)
pd_series_of_labels_bool_train = torch.cat([torch.ones(activations_true[:2000].size(0), dtype=torch.bool), torch.zeros(activations_false[:2000].size(0), dtype=torch.bool)], dim=0)
pd_series_of_labels_bool_test = torch.cat([torch.ones(activations_true[2000:2500].size(0), dtype=torch.bool), torch.zeros(activations_false[2000:2500].size(0), dtype=torch.bool)], dim=0)

In [32]:
activations_handler_train = ActivationsHandler(
  activations=activations_tensor_train, # torch tensor of [batch, activations]
  labels=pd_series_of_labels_bool_train, # each row label should correspond to the row in activations_tensor
)

activations_handler_test = ActivationsHandler(
  activations=activations_tensor_test, # torch tensor of [batch, activations]
  labels=pd_series_of_labels_bool_test, # each row label should correspond to the row in activations_tensor
)

direction_classifier, direction_calculator = get_correctness_direction_classifier(
  activations_handler_train=activations_handler_train,
  activations_handler_test=activations_handler_test,
)

In [34]:
direction_classifier.classification_metrics

{'optimal_cut': 0.6012713313102722,
 'optimal_train_set_cut': 0.6012713313102722,
 'test_roc_auc': 0.963172,
 'accuracy_score': 0.898,
 'f1_score': 0.898406374501992,
 'precision_score': 0.8948412698412699,
 'recall_score': 0.902}